In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "drive/My Drive/Colab Notebooks/cracks/models.py" .
!cp "drive/My Drive/Colab Notebooks/cracks/losses.py" .
!cp "drive/My Drive/Colab Notebooks/cracks/utils.py" .
!cp "drive/My Drive/Colab Notebooks/cracks/dataset.csv" .

In [3]:
!pip install -U segmentation-models
!pip install albumentations

     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 634kB 9.1MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=705b282c3c4627498901686e92fadd12829a9da067113096da4208f96e48255d
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [4]:
import os

import pandas as pd

import models
import losses
import utils
import json
from keras import callbacks

from matplotlib import pyplot as plt
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import train_test_split

In [5]:
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


# Обучение модели

In [6]:
INPUT_SHAPE2 = (544, 1984, 1)
#INPUT_SHAPE2 = (1056, 3936, 1) #16
#INPUT_SHAPE2 = (1056, 3888, 1) #8
#INPUT_SHAPE1 = (352, 1152, 1)
CHECKPOINT_PATH = 'drive/My Drive/Colab Notebooks/cracks/model.h5'


BATCH_SIZE = 1

In [7]:
df = pd.read_csv('dataset.csv')
LEN_OF_DATASET = df.shape[0]

## Разбиение данных на тест и проверку

In [8]:
test_idxs = [21, 65, 75, 100]
test_blur_contr = df.iloc[test_idxs]
df.drop(test_idxs, inplace = True)

train, val = train_test_split(df, test_size=0.15, random_state=666)

X_train = train['Labeled Data']
y_train = train['Label']
X_test = val['Labeled Data']
y_test = val['Label']

## Список преобразований, которые позволяют расширить датасет

In [9]:
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma, OneOf
)

from albumentations.augmentations.transforms import Blur

augs = Compose([
                    HorizontalFlip(p=0.5),
                    VerticalFlip(p=0.5),
                    OneOf([
                      RandomContrast(limit=0.2, p=0.5),
                      RandomBrightness(limit=0.4, p=0.5)
                    ],p=0.2),
                    Blur(blur_limit=5, p=0.1)
                    
                    #RandomGamma(gamma_limit=(80, 120), p=0.2),

    ])

## Инициализация генераторов данных

In [10]:
train_gen = utils.genData(X_train, y_train, BATCH_SIZE, INPUT_SHAPE2, augmentations=augs) # генератор для обучения
val_gen = utils.genData(X_test, y_test, BATCH_SIZE, INPUT_SHAPE2, augmentations=None) # генератор для валидации

## Инициализация модели
  В качестве функции потерь используется коэффицент Дайса, тк он более чувствителен к несбалансированным датасетам, также для балансировки классов используются веса классов.

In [11]:
#model = unet.simple_unet(INPUT_SHAPE2, CHECKPOINT_PATH)
#class_weights = [.15, .25, .6] 
class_weights = [.10, .33, .57]

#model = sm.Unet('resnet18',input_shape=INPUT_SHAPE2,activation='softmax',encoder_weights=None, classes=3, decoder_filters=(64, 32, 16, 8, 4))
#model = sm.PSPNet('resnet34', input_shape=INPUT_SHAPE2, classes=3, activation='softmax', encoder_weights=None, downsample_factor=16, psp_conv_filters=32, psp_dropout=0.2)
#model = unet.Unet(INPUT_SHAPE2, 3)
model = models.Unet(INPUT_SHAPE2, 3)
model.compile(RMSprop(learning_rate=3e-5), loss=sm.losses.DiceLoss(class_weights=class_weights), metrics = [sm.metrics.IOUScore(), sm.metrics.f1_score])
#model.load_weights(CHECKPOINT_PATH)

## Обучение модели

In [ ]:
h = model.fit(train_gen, validation_data=val_gen, epochs=30 ,steps_per_epoch=len(X_train) // BATCH_SIZE,
                  validation_steps=len(X_test) // BATCH_SIZE,
                  callbacks=[
                      callbacks.ModelCheckpoint(
                          filepath=CHECKPOINT_PATH,
                          monitor='val_loss',
                          mode='min',
                          save_best_only=True,
                          save_weights_only=True
                      ),
                      callbacks.EarlyStopping(
                          monitor='val_loss',
                          patience=5,
                          mode='min'
                      ),
                      callbacks.ReduceLROnPlateau(
                          patience=4,
                          monitor='val_loss',
                          mode='min'
                      )
                  ]).history

Epoch 1/30
354/355 [============================>.] - ETA: 6s - loss: 0.9330 - iou_score: 0.2764 - f1-score: 0.3659 

In [ ]:
# График с историей обученичения
plt.plot(range(1, len(h['loss']) + 1), h['loss'], label='Training DICE')
plt.plot(range(1, len(h['val_loss']) + 1), h['val_loss'], label='Val DICE')
plt.xlabel('Epochs')
plt.ylabel('DICE')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()    

plt.plot(range(1, len(h['iou_score']) + 1), h['iou_score'], label='Training Iou')
plt.plot(range(1, len(h['val_iou_score']) + 1), h['val_iou_score'], label='Val Iou')
plt.xlabel('Epochs')
plt.ylabel('mean IoU')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()    
